# Building your own Service with Awadb and ChatGLM-6B

In [1]:
import os, sys, torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel
from langchain import HuggingFacePipeline, ConversationChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import AwaDB
from langchain.document_loaders import TextLoader
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

In [2]:
Project_path = "Input Your Project path"
model_path = "THUDM/chatglm-6b"  # or you can input your local model path

# You can use `THUDM/chatglm-6b-int8` or `THUDM/chatglm-6b-int4` to reduce model size
# or you can use cpu by `AutoModel.from_pretrained(model_path, trust_remote_code=True).float()`
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModel.from_pretrained(model_path, trust_remote_code=True).half().cuda()

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

## Query model about ThreeBody 
“面壁计划”是三体书中的词汇，在没有输入知识的时候，模型并不知道面壁计划是什么，可能是训练数据中不包含三体，也可能是训练数据含有多种面壁计划产生歧义

In [8]:
query = "你好!"
response, history = model.chat(tokenizer, query, history=[])
print(f"Human: {query}\nChatGLM:{response}\n")
query = "你知道面壁计划吗，你知道面壁者都有谁吗?"
response, history = model.chat(tokenizer, query, history=history)
print(f"Human: {query}\nChatGLM:{response}\n")

Human: 你好!
ChatGLM:你好！有什么我可以帮助你的吗？

Human: 你知道面壁计划吗，你知道面壁者都有谁吗?
ChatGLM:面壁计划是指爱因斯坦在1921年提出的一个计划，旨在解决数学和物理学中的一些难题，包括狭义相对论和广义相对论等。这个计划被称为“面壁者计划”，因为它是指爱因斯坦在实验室里进行的研究，而不是指他在实际环境中进行的科学研究。

由于爱因斯坦在二战期间遭受了严重的头部创伤，他的身体健康状况逐渐恶化，他最终在1955年逝世，享年76岁。因此，“面壁者计划”通常指的是他的相对论研究计划。

在“面壁者计划”中，爱因斯坦提出了一些重要的数学和物理学理论，包括狭义相对论、广义相对论、质能方程、光电效应、波动方程等。他的理论和思想对现代物理学和数学的发展产生了深远的影响。



## Query model about ThreeBody with your local knowleadge 
由于本地训练/微调成本较高，可以采用向量数据库的形式构造本地知识库
通过结合本地知识，构成新的prompt后，模型可以根据背景知识，准确作答

In [9]:

# Add your local knowledge files
file_path = "Your local file path"
loader = TextLoader(file_path,encoding="utf-8")
documents = loader.load()

# split text into sentences and embedding the sentences
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=100,keep_separator=False)
texts = text_splitter.split_documents(documents)

embedding_path = "GanymedeNil/text2vec-large-chinese"
embeddings = HuggingFaceEmbeddings(model_name=embedding_path)
docsearch = AwaDB.from_documents(texts, embeddings,table_name="localdb")

No sentence-transformers model found with name /root/.cache/torch/sentence_transformers/GanymedeNil_text2vec-large-chinese. Creating a new one with MEAN pooling.


In [10]:
query = "你知道面壁计划吗，你知道面壁者都有谁吗?"
res = docsearch.similarity_search_with_score(query, 3)
for idx,tmp in enumerate(res[0:2]): 
    print(f"{'#'*20}第{idx+1}段相关文档{'#'*20}\n\n{tmp[0].metadata['text']}\n")

####################第1段相关文档####################

天啊！

    仅一瞬间，罗辑就悟出了面壁者这个身份的实质。正如萨伊曾说过的，这种使命在被交付前，是不可能向要承担它的人征求意见的；而面壁者的使命和身份一旦被赋予，也不可能拒绝或放弃。这种不可能并非来自于谁的强制，而是一个由面壁计划的本质所决定的冷酷逻辑，因为当一个人成为面壁者后，一层无形的不可穿透的屏障就立刻在他与普通人之间建立起来，他的一切行为就具有了面壁计划的意义，正像那对面壁者的微笑所表达的含义：

    我们怎么知道您是不是已经在工作了？

    罗辑现在终于明白，面壁者是历史上从未有过的最诡异的使命，它的逻辑冷酷而变态，但却像锁住普罗米修斯的铁环般坚固无比，这是一个不可撤销的魔咒，面壁者根本不可能凭自身的力量打破它。不管他如何挣扎，一切的一切都在对面壁者的微笑中被赋予了面壁计划的意义：

    我们怎么知道您是不是在工作？
------------

第70章 面壁者(13)

####################第2段相关文档####################

“面壁计划的核心，就是选定一批战略计划的制订者和领导者，他们完全依靠自己的思维制订战略计划，不与外界进行任何形式的交流，计划的真实战略思想、完成的步骤和最后目的都只藏在他们的大脑中，我们称他们为面壁者，这个古代东方冥思者的名称很好地反映了他们的工作特点。在领导这些战略计划执行的过程中，面壁者对外界所表现出来的思想和行为，应该是完全的假象，是经过精心策划的伪装、误导和欺骗，面壁者所要误导和欺骗的是包括敌方和己方在内的整个世界，最终建立起一个扑朔迷离的巨大的假象迷宫，使敌人在这个迷宫中丧失正确的判断，尽可能地推迟其判明我方真实战略意图的时间。

    “面壁者将被授予很高的权力，使他们能够调集和使用地球已有的战争资源中的一部分。在战略计划的执行过程中，面壁者不必对自己的行为和命令做出任何解释，不管这种行为是多么不可理解。面壁者的行为将由联合国行星防御理事会进行监督和控制，这也是唯一有权根据联合国面壁法案最后否决面壁者指令的机构。



In [11]:
# combine your local knowleadge and query 
context = "".join([tmp[0].metadata["text"] for tmp in res])
new_query = f"基于以下信息，尽可能准确的来回答用户的问题。背景信息:\n {context} \n 回答用户这个问题:{query}\n\n"
response, history = model.chat(tokenizer, new_query, history=[])
print(f"ChatGLM:{response}\n")

ChatGLM:面壁计划是指由联合国行星防御理事会选定的四位面壁者，他们的任务是领导战略计划的制订和执行，并建立一个扑朔迷离的假象迷宫，以推迟敌人判明我方真实战略意图的时间。面壁者将被授予很高的权力，调集和使用地球已有的战争资源中的一部分，但不需要对自己的行为和命令做出解释。面壁计划是一项人类历史上最艰难的使命，所有面壁者将孤独地走过漫长的岁月。



## Add new knowleadge
在数据库中增加关于awadb的相关知识，可以通过add_texts方法快速添加新知识

In [12]:
query = "你知道awadb是什么吗?"
response, history = model.chat(tokenizer, query, history=history)
print(f"Human: {query}\nChatGLM:{response}\n")

Human: 你知道awadb是什么吗?
ChatGLM:抱歉，我不知道awadb是什么。我是一个基于自然语言处理技术的人工智能助手，我的知识来自于已经存储在我的数据库中的大量文本数据。我无法浏览互联网或访问外部信息，因此我可能无法提供有关最新事件或信息的答案。如果您有任何其他问题，我将尽力回答。



In [13]:
awadb_info = ["awadb 是一款为大语言模型LLM打造的向量数据库，用于存储和快速搜索模型编码后的向量，帮助打造基于个人知识库的大模型应用",
              "awadb 支持OpenAI, Llama, Vicuna, Alpaca, ChatGLM, Dolly等模型，以及LangChain库",
              "awadb 是基于C语言开发的，并提供python接口，可以直接通过pip安装"]
docsearch.add_texts(awadb_info)

['1aa15dbd6dcdc1350b45d8be08132cb6',
 'ba84e564e2c81e6e0075d9fa92e98ebb',
 '06e572d36c121948977ef9968f553a6e']

In [18]:
query = "你知道awadb是什么吗?"
res = docsearch.similarity_search_with_score(query, 3)
for idx,tmp in enumerate(res[0:2]): 
    print(f"{'#'*20}第{idx+1}段相关文档{'#'*20}\n\n{tmp[0].metadata['text']}\n")

####################第1段相关文档####################

awadb 是一款为大语言模型LLM打造的向量数据库，用于存储和快速搜索模型编码后的向量，帮助打造基于个人知识库的大模型应用

####################第2段相关文档####################

awadb 是基于C语言开发的，并提供python接口，可以直接通过pip安装



In [19]:
context = "".join([tmp[0].metadata["text"] for tmp in res])
new_query = f"基于以下信息，尽可能准确的来回答用户的问题。背景信息:\n {context} \n 回答用户这个问题:{query}\n\n"
response, history = model.chat(tokenizer, new_query, history=[])
print(f"ChatGLM:{response}\n")

ChatGLM:是的，我知道awadb是什么。awadb是一款为大语言模型LLM打造的向量数据库，用于存储和快速搜索模型编码后的向量，帮助打造基于个人知识库的大模型应用。它是基于C语言开发的，并提供Python接口，可以直接通过pip安装。awadb支持OpenAI, Llama, Vicuna, Alpaca, ChatGLM, Dolly等模型，以及LangChain库。

